In [ ]:
# 现在导入 codecarbon，它会从指定的路径中加载
import codecarbon
print(codecarbon.__file__)  # 打印出 codecarbon 模块的路径，确保它是从新路径加载的


In [ ]:
from codecarbon.external.geography import GeoMetadata

g = GeoMetadata()           # 不传参数，内部会自动调用 IP 接口
area = g.get_grid_area()
print("国家:", g.country_name)
print("省份:", g.region)
print("经纬度:", g.latitude, g.longitude)
print("电网区域:", area)


In [42]:
import os
from codecarbon import EmissionsTracker

# 设置代理（如有需要）
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"
os.environ["ALL_PROXY"]   = "socks5h://127.0.0.1:7890"

# ===== 初始化碳排放追踪器 =====
tracker = EmissionsTracker(
    project_name="MNIST",
    output_dir='D:\\我的坚果云\\毕业设计\\codecarbon_CHN\\carbon_logs',
    measure_power_secs=1,
    tracking_mode="process",
    log_level="error",
    grid_emission_mode="om"  # 使用华北的 OM 排放因子
)

tracker.start()

# ===== MNIST 训练部分 =====
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 数据准备
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 简单模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# 训练设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 开始训练
epochs = 15
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")

# ===== 停止碳排放追踪并输出结果 =====
emissions = tracker.stop()
print(f"MNIST 训练任务碳排放总计: {emissions:.6f} kg CO₂eq")


ModuleNotFoundError: No module named 'torch'

In [ ]:
import time
from codecarbon import OfflineEmissionsTracker

# 华北 — 北京
lat, lon = 39.90, 116.40
tracker = OfflineEmissionsTracker(
    project_name="offline_huabei",
    country_iso_code="CHN",
    region="beijing",
    grid_emission_mode="om"
)
# 1) 手动指定 GeoMetadata
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
# 2) 同步到内部配置，否则输出里不会更新
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon

tracker.start()
time.sleep(1)
em_huabei = tracker.stop()

print(f"北京  (lat={lat}, lon={lon}) → 排放：{em_huabei:.6f} kgCO₂eq")


In [ ]:
# 华东 —— 上海 (31.23, 121.47)
lat, lon = 31.23, 121.47
tracker = OfflineEmissionsTracker(
    project_name="offline_huadong",
    country_iso_code="CHN",
    region="shanghai",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_huadong = tracker.stop()
print(f"华东（上海, {lat}, {lon}）→ {em_huadong:.6f} kgCO₂eq")

In [ ]:
# 华南 —— 广东（广州, 23.13, 113.26）
lat, lon = 23.13, 113.26
tracker = OfflineEmissionsTracker(
    project_name="offline_huanan",
    country_iso_code="CHN",
    region="guangdong",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_huanan = tracker.stop()
print(f"华南（广州, {lat}, {lon}）→ {em_huanan:.6f} kgCO₂eq")

In [ ]:
# 华中 —— 河南（郑州, 34.75, 113.67）
lat, lon = 34.75, 113.67
tracker = OfflineEmissionsTracker(
    project_name="offline_huazhong",
    country_iso_code="CHN",
    region="henan",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_huazhong = tracker.stop()
print(f"华中（郑州, {lat}, {lon}）→ {em_huazhong:.6f} kgCO₂eq")

In [ ]:
# 西北 —— 陕西（西安, 34.27, 108.93）
lat, lon = 34.27, 108.93
tracker = OfflineEmissionsTracker(
    project_name="offline_xibei",
    country_iso_code="CHN",
    region="shaanxi",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_xibei = tracker.stop()
print(f"西北（西安, {lat}, {lon}）→ {em_xibei:.6f} kgCO₂eq")

In [ ]:
# 西南 —— 四川（成都, 30.67, 104.06）
lat, lon = 30.67, 104.06
tracker = OfflineEmissionsTracker(
    project_name="offline_xinan",
    country_iso_code="CHN",
    region="sichuan",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_xinan = tracker.stop()
print(f"西南（成都, {lat}, {lon}）→ {em_xinan:.6f} kgCO₂eq")

In [ ]:
# 东北 —— 辽宁（沈阳, 41.80, 123.43）
lat, lon = 41.80, 123.43
tracker = OfflineEmissionsTracker(
    project_name="offline_dongbei",
    country_iso_code="CHN",
    region="liaoning",
    grid_emission_mode="om"
)
tracker._geo.latitude  = lat
tracker._geo.longitude = lon
tracker._conf["latitude"]  = lat
tracker._conf["longitude"] = lon
tracker.start()
time.sleep(1)
em_dongbei = tracker.stop()
print(f"东北（沈阳, {lat}, {lon}）→ {em_dongbei:.6f} kgCO₂eq")